Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/home/dmitry/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) 
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.238813
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.564392
Minibatch accuracy: 56.2%
Validation accuracy: 50.7%
Minibatch loss at step 100: 0.967773
Minibatch accuracy: 68.8%
Validation accuracy: 68.1%
Minibatch loss at step 150: 0.564826
Minibatch accuracy: 81.2%
Validation accuracy: 76.0%
Minibatch loss at step 200: 0.343225
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 250: 1.512059
Minibatch accuracy: 62.5%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.851346
Minibatch accuracy: 75.0%
Validation accuracy: 77.4%
Minibatch loss at step 350: 0.621909
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Minibatch loss at step 400: 0.606625
Minibatch accuracy: 75.0%
Validation accuracy: 80.1%
Minibatch loss at step 450: 0.497424
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 500: 0.084799
Minibatch accuracy: 100.0%
Validation accuracy: 80.9%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

With two pooling layers my computer gives up :D

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
image_size = 28
num_channels = 1

graph = tf.Graph()
with graph.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Weights
    W_1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    b_1 = tf.Variable(tf.zeros([depth]))
    W_2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    b_2 = tf.Variable(tf.constant(1.0, shape=[depth]))
    W_3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    b_3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    W_4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    b_4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    def model(data):
        conv_1 = tf.nn.conv2d(data, W_1, [1, 1, 1, 1], padding='SAME')
        relu_1 = tf.nn.relu(conv_1 + b_1)
        mpool_1 = tf.nn.max_pool(relu_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        conv_2 = tf.nn.conv2d(mpool_1, W_2, [1, 2, 2, 1], padding='SAME')
        relu_2 = tf.nn.relu(conv_2 + b_2)
        shape = relu_2.get_shape().as_list()
        reshape = tf.reshape(relu_2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, W_3) + b_3)
        return tf.matmul(hidden, W_4) + b_4
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.901646
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.605212
Minibatch accuracy: 56.2%
Validation accuracy: 52.2%
Minibatch loss at step 100: 0.993238
Minibatch accuracy: 62.5%
Validation accuracy: 68.6%
Minibatch loss at step 150: 0.656348
Minibatch accuracy: 87.5%
Validation accuracy: 75.4%
Minibatch loss at step 200: 0.232606
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 250: 1.317038
Minibatch accuracy: 68.8%
Validation accuracy: 79.0%
Minibatch loss at step 300: 0.894229
Minibatch accuracy: 68.8%
Validation accuracy: 78.0%
Minibatch loss at step 350: 0.434511
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 400: 0.675660
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 450: 0.651609
Minibatch accuracy: 75.0%
Validation accuracy: 81.9%
Minibatch loss at step 500: 0.209344
Minibatch accuracy: 93.8%
Validation accuracy: 81.1%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

For more sophisticated architectures I need a more powerful computer. (I'll try to improve the result later)

In [9]:
batch_size = 32
patch_size = 5
depth_1 = 8
depth_2 = 16
num_hidden = 128
image_size = 28
num_channels = 1

graph = tf.Graph()
with graph.as_default():
    
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Weights
    W_1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth_1], stddev=0.1))
    b_1 = tf.Variable(tf.zeros([depth_1]))
    W_2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth_1, depth_2], stddev=0.1))
    b_2 = tf.Variable(tf.constant(1.0, shape=[depth_2]))
    W_3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth_2, num_hidden], stddev=0.1))
    b_3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    W_4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    b_4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    def model(data, drop=True):
        conv_1 = tf.nn.conv2d(data, W_1, [1, 1, 1, 1], padding='SAME')
        relu_1 = tf.nn.relu(conv_1 + b_1)
        mpool_1 = tf.nn.max_pool(relu_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        conv_2 = tf.nn.conv2d(mpool_1, W_2, [1, 1, 1, 1], padding='SAME')
        relu_2 = tf.nn.relu(conv_2 + b_2)
        mpool_2 = tf.nn.max_pool(relu_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = mpool_2.get_shape().as_list()
        reshape = tf.reshape(mpool_2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, W_3) + b_3)
        if drop:
            dropout = tf.nn.dropout(hidden, 0.5)
            return tf.matmul(dropout, W_4) + b_4
        else:
            return tf.matmul(hidden, W_4) + b_4
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, drop=False))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, drop=False))

In [10]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 200 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.433690
Minibatch accuracy: 9.4%
Validation accuracy: 14.1%
Minibatch loss at step 200: 1.082455
Minibatch accuracy: 68.8%
Validation accuracy: 76.2%
Minibatch loss at step 400: 0.717629
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 600: 0.794665
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Minibatch loss at step 800: 0.915264
Minibatch accuracy: 78.1%
Validation accuracy: 82.5%
Minibatch loss at step 1000: 0.532624
Minibatch accuracy: 90.6%
Validation accuracy: 83.8%
Minibatch loss at step 1200: 0.927424
Minibatch accuracy: 75.0%
Validation accuracy: 84.0%
Minibatch loss at step 1400: 0.874402
Minibatch accuracy: 75.0%
Validation accuracy: 84.4%
Minibatch loss at step 1600: 0.526962
Minibatch accuracy: 84.4%
Validation accuracy: 85.3%
Minibatch loss at step 1800: 0.629238
Minibatch accuracy: 81.2%
Validation accuracy: 85.6%
Minibatch loss at step 2000: 0.591611
Minibatch accuracy: 75.0%
Validation accuracy: 8